In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import  Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# Checking if we able to send emails

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email=Email("hannahabraham20@gmail.com")
    to_email=To("hannah.abraham87@gmail.com")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Agent Workflow

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent_1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent_2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent_3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

In [6]:
result = Runner.run_streamed(sales_agent_1,input="Write a cold sales email")
async for event in result.stream_events():
    if event.type =="raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
        print(event.data.delta, end="",flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we are dedicated to streamlining the SOC 2 compliance process for businesses like yours.

Navigating SOC 2 audits can be complex and time-consuming. ComplAI’s AI-powered SaaS tool simplifies this journey by providing a comprehensive framework to ensure compliance while saving you valuable time and resources. Our platform not only assists in maintaining compliance but also prepares your team for successful audits with minimal disruption to your operations.

Here are a few key benefits our solution offers:

- **Automated Compliance Tracking**: Stay updated with real-time compliance status and notifications.
- **User-Friendly Interface**: Simplified dashboards make it easy for your team to engage with compliance requirements.
- **Expert Guidance**: Access to compliance templates and documentation, tailored to you

In [7]:
message ="Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1,message),
        Runner.run(sales_agent_2,message),
        Runner.run(sales_agent_3,message)
    )

In [8]:
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance can be challenging, especially as your organization continues to grow. At ComplAI, we specialize in streamlining the entire compliance process through our innovative SaaS tool, which leverages AI to ensure you meet all audit requirements efficiently.

Our solution not only helps you maintain ongoing compliance but also prepares you for audits with minimal disruption to your operations. By automating many of the time-consuming tasks, we empower your team to focus on what truly matters—driving your business forward.

I'd love to schedule a brief call to discuss how ComplAI can support your organization's compliance efforts and help you achieve peace of mind. Please let me know a convenient time for you, or feel free to book directly on my calendar [insert link].

Thank you for considering ComplAI. I look forward to th

In [9]:
sales_picker = Agent(
    name = "sales_picker",
instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
model="gpt-4o-mini")

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


### Tools

In [ ]:
@function_tool
def send_email_body(body:str):
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email=Email("hannahabraham20@gmail.com")
    to_email=To("hannah.abraham87@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

In [ ]:
tool1 = sales_agent_1.as_tool(tool_name="sales_agent_1",tool_description="Write a cold sales email")
tool1

In [ ]:
description = "Write a cold sales email"
tool1 = sales_agent_1.as_tool(tool_name="sales_agent_1",tool_description=description)
tool2 = sales_agent_2.as_tool(tool_name="sales_agent_2",tool_description=description)
tool3 = sales_agent_3.as_tool(tool_name="sales_agent_3",tool_description=description)


In [ ]:
tools =[tool1,tool2,tool3,send_email_body]
tools

## Sales Manager  - Our Planning agent

In [ ]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

sales_manager = Agent(
    name = "sales_manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager,message)


## Handoffs

In [ ]:
subject_instructions="You can write a subject for a cold sales email.\
    You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions="You convert a text email body to an html email body \
    You are given a text email body which might have some markdown\
        and you need to convert it to an HTML email body with simple, clear, compelling layout and design"

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool=subject_writer.as_tool(tool_name="subject_writer",tool_description="Write a subject for a cold sales email")

html_writer = Agent(name="HTML email body convertor", instructions=subject_instructions, model="gpt-4o-mini")
html_tool=html_writer.as_tool(tool_name="html_writer",tool_description="WConvert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("hannahabraham20@gmail.com")  
    to_email = To("hannah.abraham87@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools =[subject_tool,html_tool,send_html_email]
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(name="Email manager", instructions=instructions, tools=tools, model="gpt-4o-mini", handoff_description="Convert an email to HTML and send it")

In [ ]:

tools = [tool1,tool2,tool3]
handoffs =[emailer_agent]
print(tools)
print(handoffs)

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

sales_manager =Agent(
    name="Sales Manager",
    tools=tools,
    instructions=sales_manager_instructions,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"


with trace("Automated SDR"):
    result =await Runner.run(sales_manager,message)